In [ ]:
import sys
sys.path.insert(0,'../scripts')
from pycap_for_PESTPP_MOU import postprocess_MOU, plot_pareto, prep_for_viz, create_viz_app, plot_pareto_with_scenarios
from pathlib import Path
from ipywidgets import interact, fixed
import yaml
import pandas as pd
import pyemu



# Read in one of the configuration files and postprocess it. 
### The `config_file` should be your `secnario_name` + `".yml"` as assigned in the `05_MOU_setup_run.ipynb` notebook. 

In [ ]:
config_file = 'fish_cash_baseline.yml'

with open(Path('./configurations') / config_file,'r') as ifp:
    inpars = yaml.safe_load(ifp)
inpars

### Below, the `config_file` is processed, beginning the postprocessing

In [ ]:
run_path = Path(inpars['run_path'])
run_name = inpars['scenario_name']
pareto_df = postprocess_MOU(run_name, run_path)

In [ ]:
pareto_df= pareto_df.loc[pareto_df.generation==50]
pareto_df

## Let's scrape through all the results and see what happens if we also evaluate total pumping and streamflow objectives

In [ ]:
# get all the observations files - beware of some python shenanigans here!
# this reads in the observations for ALL model simulations - probably more than 10,000
obs_df = pd.concat([pd.read_csv(i, index_col=0)
                   for i in run_path.glob("*obs_pop*")])


In [ ]:
# the three columns we want are "ag_receipts","fish_prob", and "lpr:total_combined:bdpl" (that last one is the total depletion)
# and, we can use the indices from the pareto_df to keep track of the final realizations
obs_df.index = obs_df.index.astype(str)
obs_df = obs_df.loc[pareto_df.member,
                    ["ag_receipts","fish_prob", "lpr:total_combined:bdpl"]].drop_duplicates()

In [ ]:
obs_df

In [ ]:
# similarly for pumping
dv_df = pd.concat([pd.read_csv(i, index_col=0)
                   for i in run_path.glob("*dv_pop*")]).loc[obs_df.index].drop_duplicates()

In [ ]:
dv_df

In [ ]:
#Tricky thing is, for receipts case, if pumping drops below 70% of the original value, 
# it goes to zero, so let's adjust
# first we need those original pumping values
pst = pyemu.Pst(str(run_path / f"{run_name}.pst"))
pars = pst.parameter_data

In [ ]:
pars

In [ ]:
dv_df.sum(axis=1)

In [ ]:
# next for each well, we need to adjust, but only of "dollar" is in the objectives
dollars=False
if 'dollar' in inpars['objectives']:
    dollars = True
if dollars is True:
    for cc in dv_df.columns:
        dv_df.loc[dv_df[cc]<=pars.loc[cc,'parval1']*.7,cc]=0


#### finally, sum up all the pumping and the result is 4 columns that can be plotted against each other

In [ ]:
obs_df['pumping_sum'] = dv_df.sum(axis=1).loc[obs_df.index]

In [ ]:
obs_df